# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [ ]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [ ]:
def answer_one():
    spam_percentage = (spam_data['target'].sum() / len(spam_data)) * 100
    return spam_percentage

answer_one()


### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer()
    X_train_transformed = vectorizer.fit_transform(X_train)
    vocab = vectorizer.get_feature_names_out()
    longest_token = max(vocab, key=len)
    return longest_token

answer_two()


### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vectorizer = CountVectorizer()
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_test_transformed = vectorizer.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_transformed, y_train)
    y_probs = model.predict_proba(X_test_transformed)[:, 1]
    
    auc_score = roc_auc_score(y_test, y_probs)
    return auc_score

answer_three()


### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def answer_four():
    vectorizer = TfidfVectorizer()
    X_train_transformed = vectorizer.fit_transform(X_train)
    
    feature_names = vectorizer.get_feature_names_out()
    max_tf_idf = X_train_transformed.max(axis=0).toarray().flatten()
    
    tfidf_df = pd.DataFrame({'feature': feature_names, 'tfidf': max_tf_idf})
    tfidf_df = tfidf_df.sort_values(['tfidf', 'feature'])
    
    smallest_tf_idf = tfidf_df.head(20).set_index('feature')['tfidf']
    largest_tf_idf = tfidf_df.tail(20).sort_values(['tfidf', 'feature'], ascending=[False, True]).set_index('feature')['tfidf']
    
    return (smallest_tf_idf, largest_tf_idf)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_five():
    vectorizer = TfidfVectorizer(min_df=3)
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_test_transformed = vectorizer.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_transformed, y_train)
    y_probs = model.predict_proba(X_test_transformed)[:, 1]
    
    auc_score = roc_auc_score(y_test, y_probs)
    return auc_score

answer_five()


### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [ ]:
def answer_six():
    spam_lengths = spam_data[spam_data['target'] == 1]['text'].apply(len)
    not_spam_lengths = spam_data[spam_data['target'] == 0]['text'].apply(len)
    
    avg_length_spam = spam_lengths.mean()
    avg_length_not_spam = not_spam_lengths.mean()
    
    return (avg_length_not_spam, avg_length_spam)

answer_six()


<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

def answer_seven():
    vectorizer = TfidfVectorizer(min_df=5)
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_test_transformed = vectorizer.transform(X_test)
    
    # Add document length feature
    X_train_transformed = add_feature(X_train_transformed, X_train.apply(len))
    X_test_transformed = add_feature(X_test_transformed, X_test.apply(len))
    
    model = SVC(C=10000, probability=False)
    model.fit(X_train_transformed, y_train)
    y_scores = model.decision_function(X_test_transformed)
    
    auc_score = roc_auc_score(y_test, y_scores)
    return auc_score

answer_seven()


### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [ ]:
import re

def count_digits(text):
    return len(re.findall(r'\d', text))

def answer_eight():
    spam_digit_counts = spam_data[spam_data['target'] == 1]['text'].apply(count_digits)
    not_spam_digit_counts = spam_data[spam_data['target'] == 0]['text'].apply(count_digits)
    
    avg_digits_spam = spam_digit_counts.mean()
    avg_digits_not_spam = not_spam_digit_counts.mean()
    
    return (avg_digits_not_spam, avg_digits_spam)

answer_eight()


### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack, csr_matrix

def count_digits(text):
    return sum(c.isdigit() for c in text)

def answer_nine():
    # TfidfVectorizer with specified parameters
    vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_test_transformed = vectorizer.transform(X_test)
    
    # Additional features
    X_train_length = X_train.str.len().values.reshape(-1, 1)
    X_test_length = X_test.str.len().values.reshape(-1, 1)
    
    X_train_digit_count = X_train.apply(count_digits).values.reshape(-1, 1)
    X_test_digit_count = X_test.apply(count_digits).values.reshape(-1, 1)
    
    # Convert to sparse matrices
    X_train_length = csr_matrix(X_train_length)
    X_test_length = csr_matrix(X_test_length)
    X_train_digit_count = csr_matrix(X_train_digit_count)
    X_test_digit_count = csr_matrix(X_test_digit_count)
    
    # Stack features
    X_train_combined = hstack([X_train_transformed, X_train_length, X_train_digit_count])
    X_test_combined = hstack([X_test_transformed, X_test_length, X_test_digit_count])
    
    # Fit Logistic Regression model
    model = LogisticRegression(C=100, max_iter=1000)
    model.fit(X_train_combined, y_train)
    
    # Predict probabilities
    y_probs = model.predict_proba(X_test_combined)[:, 1]
    
    # Calculate AUC score
    auc_score = roc_auc_score(y_test, y_probs)
    
    return auc_score

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
def count_non_word_chars(text):
    return len(re.findall(r'\W', text))

def answer_ten():
    spam_non_word_counts = spam_data[spam_data['target'] == 1]['text'].apply(count_non_word_chars)
    not_spam_non_word_counts = spam_data[spam_data['target'] == 0]['text'].apply(count_non_word_chars)
    
    avg_non_word_chars_spam = spam_non_word_counts.mean()
    avg_non_word_chars_not_spam = not_spam_non_word_counts.mean()
    
    return (avg_non_word_chars_not_spam, avg_non_word_chars_spam)

answer_ten()


### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack, csr_matrix
import numpy as np

def answer_eleven():
    # Use only first 2000 rows of training data
    X_train_2000 = X_train[:2000]
    y_train_2000 = y_train[:2000]
    
    vectorizer = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5))
    X_train_transformed = vectorizer.fit_transform(X_train_2000)
    X_test_transformed = vectorizer.transform(X_test)
    
    # Additional features
    X_train_length = X_train_2000.str.len().values.reshape(-1, 1)
    X_test_length = X_test.str.len().values.reshape(-1, 1)
    
    X_train_digit_count = X_train_2000.apply(lambda x: sum(c.isdigit() for c in x)).values.reshape(-1, 1)
    X_test_digit_count = X_test.apply(lambda x: sum(c.isdigit() for c in x)).values.reshape(-1, 1)
    
    X_train_non_word_count = X_train_2000.apply(lambda x: sum(not c.isalnum() and c != '_' for c in x)).values.reshape(-1, 1)
    X_test_non_word_count = X_test.apply(lambda x: sum(not c.isalnum() and c != '_' for c in x)).values.reshape(-1, 1)
    
    # Combine features
    X_train_combined = hstack([
        X_train_transformed,
        csr_matrix(X_train_length),
        csr_matrix(X_train_digit_count),
        csr_matrix(X_train_non_word_count)
    ])
    X_test_combined = hstack([
        X_test_transformed,
        csr_matrix(X_test_length),
        csr_matrix(X_test_digit_count),
        csr_matrix(X_test_non_word_count)
    ])
    
    # Fit model
    model = LogisticRegression(C=100, max_iter=1000)
    model.fit(X_train_combined, y_train_2000)
    
    # Calculate AUC
    y_scores = model.decision_function(X_test_combined)
    auc_score = roc_auc_score(y_test, y_scores)
    
    # Get coefficients
    feature_names = np.concatenate([
        vectorizer.get_feature_names_out(),
        ['length_of_doc', 'digit_count', 'non_word_char_count']
    ])
    coefficients = model.coef_[0]
    
    coef_dict = dict(zip(feature_names, coefficients))
    sorted_coefs = sorted(coef_dict.items(), key=lambda x: (x[1], x[0]))
    
    smallest_coefs = [coef for _, coef in sorted_coefs[:10]]
    largest_coefs = [coef for _, coef in sorted_coefs[-10:][::-1]]
    
    return (auc_score, smallest_coefs, largest_coefs)